## 1. Setup & Configuration

In [1]:
import sys
import os
import yaml

# Windows compatibility fix for PySpark
if sys.platform == "win32":
    import socketserver
    if not hasattr(socketserver, 'UnixStreamServer'):
        socketserver.UnixStreamServer = socketserver.TCPServer

from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Load configuration
with open("../../../../config/config.yml") as f:
    config = yaml.safe_load(f)

silver_dir = config["paths"]["silver_dir"]

print(f"✓ Silver directory: {silver_dir}")

✓ Silver directory: data/silver


In [2]:
# Initialize Spark session (Windows configuration)
import tempfile

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

local_temp = tempfile.gettempdir()
os.environ['SPARK_LOCAL_DIRS'] = local_temp

spark = SparkSession.builder \
    .appName("Add_Centroids") \
    .master("local[*]") \
    .config("spark.sql.adaptive.enabled", "false") \
    .config("spark.driver.host", "localhost") \
    .config("spark.driver.bindAddress", "localhost") \
    .config("spark.ui.enabled", "false") \
    .config("spark.executor.memory", "2g") \
    .config("spark.driver.memory", "2g") \
    .config("spark.local.dir", local_temp) \
    .config("spark.sql.shuffle.partitions", "4") \
    .getOrCreate()

spark.sparkContext.setLogLevel("WARN")

print(f"✓ Spark version: {spark.version}")

✓ Spark version: 3.5.3


## 2. Charger les données

In [3]:
# Charger l'ANCIEN silver_amenagements (Parquet)
old_path = f"../../../../{silver_dir}/silver_amenagements"

df_old = spark.read.parquet(old_path)

print(f"📦 Ancien silver_amenagements: {df_old.count()} rows")
print(f"   Colonnes: {df_old.columns}")

📦 Ancien silver_amenagements: 4592 rows
   Colonnes: ['amenagement_id', 'nom', 'commune1', 'insee1', 'commune2', 'insee2', 'reseau', 'financementac', 'typeamenagement', 'typeamenagement2', 'positionnement', 'senscirculation', 'environnement', 'localisation', 'typologiepiste', 'revetementpiste', 'domanialite', 'reglementation', 'zonecirculationapaisee', 'anneelivraison', 'longueur_m', 'observation', 'is_valid']


In [4]:
# Charger le NOUVEAU silver_amenagements (CSV avec centroids)
new_csv_path = "../../../../data/silver_amenagements.csv"

df_new = spark.read.csv(new_csv_path, header=True, inferSchema=True, sep=";")

print(f"📄 Nouveau silver_amenagements.csv: {df_new.count()} rows")
print(f"   Colonnes: {df_new.columns}")

📄 Nouveau silver_amenagements.csv: 4592 rows
   Colonnes: ['nom', 'commune1', 'insee1', 'commune2', 'insee2', 'reseau', 'financementac', 'typeamenagement', 'typeamenagement2', 'positionnement', 'senscirculation', 'environnement', 'localisation', 'typologiepiste', 'revetementpiste', 'domanialite', 'reglementation', 'zonecirculationapaisee', 'anneelivraison', 'longueur', 'observation', 'validite', 'gid', 'centroid_lat', 'centroid_lon', 'is_mock', 'geocoded_at']


In [5]:
# Vérifier que les colonnes centroid existent dans le nouveau fichier
assert "centroid_lat" in df_new.columns, "centroid_lat manquant dans le nouveau fichier!"
assert "centroid_lon" in df_new.columns, "centroid_lon manquant dans le nouveau fichier!"

print("✓ Colonnes centroid_lat et centroid_lon présentes")

# Preview des centroids
df_new.select("nom", "centroid_lat", "centroid_lon").show(5)

✓ Colonnes centroid_lat et centroid_lon présentes
+--------------------+------------+------------+
|                 nom|centroid_lat|centroid_lon|
+--------------------+------------+------------+
|Chemin des Grenettes|   45.918023|   4.7587563|
|Rond-point des Mo...|   45.954282|    4.723643|
|    Promenade Lénine|   45.772741|    4.966703|
|Piste parallèle a...|   45.749602|    4.721187|
|8ème Rue Cité Ber...|    45.73042|    4.795658|
+--------------------+------------+------------+
only showing top 5 rows



## 3. Identifier la clé de jointure

In [6]:
# Trouver la clé de jointure commune
common_cols = set(df_old.columns) & set(df_new.columns)
print(f"Colonnes communes: {common_cols}")

# Utiliser 'gid' comme clé si disponible, sinon 'nom'
if "gid" in common_cols:
    join_key = "gid"
elif "amenagement_id" in common_cols:
    join_key = "amenagement_id"
elif "nom" in common_cols:
    join_key = "nom"
else:
    raise ValueError("Aucune clé de jointure trouvée!")

print(f"✓ Clé de jointure: {join_key}")

Colonnes communes: {'typeamenagement', 'insee1', 'nom', 'positionnement', 'revetementpiste', 'commune1', 'zonecirculationapaisee', 'anneelivraison', 'environnement', 'commune2', 'reglementation', 'observation', 'typeamenagement2', 'senscirculation', 'financementac', 'localisation', 'insee2', 'reseau', 'typologiepiste', 'domanialite'}
✓ Clé de jointure: nom


## 4. Joindre les colonnes centroid

In [7]:
# Extraire uniquement les colonnes nécessaires du nouveau fichier
df_centroids = df_new.select(
    col(join_key),
    col("centroid_lat").alias("new_centroid_lat"),
    col("centroid_lon").alias("new_centroid_lon")
)

print(f"✓ Extrait {df_centroids.count()} lignes avec centroids")
df_centroids.show(5)

✓ Extrait 4592 lignes avec centroids
+--------------------+----------------+----------------+
|                 nom|new_centroid_lat|new_centroid_lon|
+--------------------+----------------+----------------+
|Chemin des Grenettes|       45.918023|       4.7587563|
|Rond-point des Mo...|       45.954282|        4.723643|
|    Promenade Lénine|       45.772741|        4.966703|
|Piste parallèle a...|       45.749602|        4.721187|
|8ème Rue Cité Ber...|        45.73042|        4.795658|
+--------------------+----------------+----------------+
only showing top 5 rows



In [8]:
# Vérifier si l'ancien a déjà les colonnes centroid
has_old_centroid = "centroid_lat" in df_old.columns

if has_old_centroid:
    print("⚠️  L'ancien fichier a déjà des colonnes centroid - elles seront remplacées")
    df_old_clean = df_old.drop("centroid_lat", "centroid_lon")
else:
    print("✓ L'ancien fichier n'a pas de colonnes centroid - elles seront ajoutées")
    df_old_clean = df_old

✓ L'ancien fichier n'a pas de colonnes centroid - elles seront ajoutées


In [9]:
# Effectuer la jointure
df_merged = df_old_clean.join(
    df_centroids,
    on=join_key,
    how="left"
).withColumnRenamed("new_centroid_lat", "centroid_lat") \
 .withColumnRenamed("new_centroid_lon", "centroid_lon")

print(f"✓ Jointure effectuée: {df_merged.count()} rows")
print(f"✓ Nouvelles colonnes: {df_merged.columns}")

✓ Jointure effectuée: 17825 rows
✓ Nouvelles colonnes: ['nom', 'amenagement_id', 'commune1', 'insee1', 'commune2', 'insee2', 'reseau', 'financementac', 'typeamenagement', 'typeamenagement2', 'positionnement', 'senscirculation', 'environnement', 'localisation', 'typologiepiste', 'revetementpiste', 'domanialite', 'reglementation', 'zonecirculationapaisee', 'anneelivraison', 'longueur_m', 'observation', 'is_valid', 'centroid_lat', 'centroid_lon']


In [10]:
# Vérifier les résultats
print("Preview des données fusionnées:")
df_merged.select(join_key, "centroid_lat", "centroid_lon").show(10)

# Compter les valeurs nulles
null_count = df_merged.filter(col("centroid_lat").isNull()).count()
print(f"\n⚠️  Lignes sans centroid: {null_count}")

Preview des données fusionnées:
+--------------------+------------+------------+
|                 nom|centroid_lat|centroid_lon|
+--------------------+------------+------------+
|Chemin des Grenettes|   45.918023|   4.7587563|
|Rond-point des Mo...|   45.954282|    4.723643|
|    Promenade Lénine|   45.775361|    4.833497|
|    Promenade Lénine|   45.712984|    4.700469|
|    Promenade Lénine|   45.684687|    4.760462|
|    Promenade Lénine|   45.772741|    4.966703|
|Piste parallèle a...|   45.749602|    4.721187|
|8ème Rue Cité Ber...|    45.73042|    4.795658|
|   Rue des Collières|   45.762876|    4.732809|
|   Rue des Collières|    45.91378|    4.872991|
+--------------------+------------+------------+
only showing top 10 rows


⚠️  Lignes sans centroid: 5


## 5. Sauvegarder le résultat

In [11]:
# Sauvegarder en Parquet (écrase l'ancien)
output_path = f"../../../../{silver_dir}/silver_amenagements3"

print(f"💾 Sauvegarde vers: {output_path}")
df_merged.write.mode("overwrite").parquet(output_path)

print("✓ Sauvegarde terminée!")

💾 Sauvegarde vers: ../../../../data/silver/silver_amenagements3
✓ Sauvegarde terminée!


In [12]:
# Vérification finale
df_verify = spark.read.parquet(output_path)

print(f"✓ Fichier relu: {df_verify.count()} rows")
print(f"✓ Colonnes: {df_verify.columns}")

assert "centroid_lat" in df_verify.columns, "centroid_lat manquant!"
assert "centroid_lon" in df_verify.columns, "centroid_lon manquant!"

print("\n🎉 Colonnes centroid ajoutées avec succès!")
df_verify.select(join_key, "centroid_lat", "centroid_lon").show(5)

✓ Fichier relu: 17825 rows
✓ Colonnes: ['nom', 'amenagement_id', 'commune1', 'insee1', 'commune2', 'insee2', 'reseau', 'financementac', 'typeamenagement', 'typeamenagement2', 'positionnement', 'senscirculation', 'environnement', 'localisation', 'typologiepiste', 'revetementpiste', 'domanialite', 'reglementation', 'zonecirculationapaisee', 'anneelivraison', 'longueur_m', 'observation', 'is_valid', 'centroid_lat', 'centroid_lon']

🎉 Colonnes centroid ajoutées avec succès!
+--------------------+------------+------------+
|                 nom|centroid_lat|centroid_lon|
+--------------------+------------+------------+
|Chemin des Grenettes|   45.918023|   4.7587563|
|Rond-point des Mo...|   45.954282|    4.723643|
|    Promenade Lénine|   45.775361|    4.833497|
|    Promenade Lénine|   45.712984|    4.700469|
|    Promenade Lénine|   45.684687|    4.760462|
+--------------------+------------+------------+
only showing top 5 rows



## 6. Cleanup

In [13]:
# Stop Spark session
spark.stop()
print("✓ Spark session stopped")

✓ Spark session stopped
